In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

# Read dataset
#path = '/kaggle/input/muse-the-musical-sentiment-dataset/muse_v3.csv'
path = '../muse_v3.csv'
data = pd.read_csv(path).astype({'valence_tags': np.float32, 'arousal_tags':np.float32, 'dominance_tags':np.float32})
train_set = data[0:60000]
test_set = data[60000:]
training_points = 20000
testing_points = 500    # 100 gave an accuracy of approx 85%
training = train_set.sample(training_points).to_numpy()
training_features = training[:, 5:8]
training_targets = training[:, 3]
testing = test_set.sample(testing_points).to_numpy()
testing_features = testing[:, 5:8]
testing_targets = testing[:, 3]

components = 276    # Number of unique genre/seed values in dataset

print('Finished loading and splitting data')

Finished loading and splitting data


In [9]:
gmm_model = GaussianMixture(n_components=components).fit(training_features)
gmm_predictions = gmm_model.predict(testing_features)

gmm_mean_to_genre = {}
for i in range(components):
    gmm_group = testing[gmm_predictions == i]
    if len(gmm_group) != 0:
        group_targets = gmm_group[:, 3]
        values, counts = np.unique(group_targets, return_counts=True)
        group_val = values[np.argmax(counts)]
        gmm_mean_to_genre[i] = group_val

gmm_correct = 0
for i in range(testing_points):
    if gmm_mean_to_genre[gmm_predictions[i]] in testing_targets[i]:
        gmm_correct += 1
gmm_accuracy = gmm_correct / testing_points   
print('GMM: ' + str(gmm_accuracy))

GMM: 0.514


In [10]:
kmm_model = KMeans(n_clusters=components).fit(training_features)
kmm_predictions = kmm_model.predict(testing_features)

kmm_mean_to_genre = {}
for i in range(components):
    kmm_group = testing[kmm_predictions == i]
    if len(kmm_group) != 0:
        group_targets = kmm_group[:, 3]
        values, counts = np.unique(group_targets, return_counts=True)
        group_val = values[np.argmax(counts)]
        kmm_mean_to_genre[i] = group_val
        
kmm_correct = 0
for i in range(testing_points):
    if kmm_mean_to_genre[kmm_predictions[i]] in testing_targets[i]:
        kmm_correct += 1
kmm_accuracy = kmm_correct / testing_points
print('KMM: ' + str(kmm_accuracy))

KMM: 0.612


In [ ]:
### Not properly implemented

temp = data.to_numpy()[:, 3]
flat_temp = np.array([], dtype=str)
for i in range(len(temp)):
    seed = temp[i][1:-1]
    flat_temp = np.append(flat_temp, np.array(seed.split(', ')))
unique_labels = np.unique(flat_temp)
print(len(unique_labels))

knn_training_targets = np.empty(training_targets.shape[0])
for i in range(knn_training_targets.shape[0]):
    labels = training_targets[i][1:-1].split(', ')
    for label in labels:
        knn_training_targets[i] += np.where(unique_labels == label)
    knn_training_targets[i] /= len(labels)
    knn_training_targets[i] = round(knn_training_targets[i])
print(knn_training_targets)

        
# knn_mean_to_genre = {}
# for i in range(components):
#     knn_group = testing[knn_predictions == i]
#     if len(knn_group) != 0:
#         group_targets = knn_group[:, 3]
#         values, counts = np.unique(group_targets, return_counts=True)
#         group_val = values[np.argmax(counts)]
#         knn_mean_to_genre[i] = group_val
        
#print(mean_to_genre.values())

knn_model = KNeighborsClassifier(n_neighbors=1, weights='distance').fit(training_features, knn_training_targets)
knn_predictions = knn_model.predict(testing_features)

knn_correct = 0
for i in range(testing_points):
    if unique_labels[round(knn_predictions[i])] in testing_targets[i]:
        knn_correct += 1
                
# for i in range(testing_points):
#     for genre in knn_predictions[i][1:-1].split(', '):
#         if genre in testing_targets[i]:
#             knn_correct += 1
#             break

knn_accuracy = knn_correct / testing_points
print('KNN: ' + str(knn_accuracy))